In [ ]:
import sys
sys.path.append(r'C:\Users\lwang\Documents\GitHub\DataAnalysis')

In [ ]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
import scipy
import importlib
import pickle

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
import importlib.util

# Set up environment (e.g. sys.path)
assembler_folder = r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler'
top_folder = r'C:\Users\lwang\Documents\GitHub\DataAnalysis'

# Set up the module function to be tested. This is equivalent to from X.Y.Z import module_func, but avoid potential ImportError
background_func_file_path = os.path.join(assembler_folder,'Modules','glucoseModel','channel_gluc_current','time_varying_background.py')
background_func_name = os.path.relpath(background_func_file_path,top_folder).replace('.py','').replace(os.sep,'.')
background_func_spec = importlib.util.spec_from_file_location(background_func_name,background_func_file_path)
background_func_info = importlib.util.module_from_spec(background_func_spec)
background_func_spec.loader.exec_module(background_func_info)
background_func = getattr(background_func_info,'background_gamma_cdf_based')

sensitivity_func_file_path = os.path.join(assembler_folder,'Modules','glucoseModel','channel_dermis_gluc','time_varying_sensitivity.py')
sensitivity_func_name = os.path.relpath(sensitivity_func_file_path,top_folder).replace('.py','').replace(os.sep,'.')
sensitivity_func_spec = importlib.util.spec_from_file_location(sensitivity_func_name,sensitivity_func_file_path)
sensitivity_func_info = importlib.util.module_from_spec(sensitivity_func_spec)
sensitivity_func_spec.loader.exec_module(sensitivity_func_info)
sensitivity_func = getattr(sensitivity_func_info,'sensitivity_gamma_cdf_based')

load data

In [ ]:
df_orig=pd.read_csv(r'A:\Algo\Isabella_v02 model comparison\agg_sample_df_ref_training_validation.csv')

In [ ]:
df_orig['ysi_acck'] = df_orig['ysi']
nan_idx = df_orig['ysi'].isna()
df_orig.loc[nan_idx,'ysi_acck'] = df_orig.loc[nan_idx,'acck']

In [ ]:
sensor_info_df = pd.read_excel(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Investigate May Study\all study sensor info.xlsx')

In [ ]:
df_orig = df_orig.merge(right=sensor_info_df,how='left',left_on='Sensor_Id',right_on='sensor_id',suffixes=(None,'2'))

In [ ]:
df_orig['Sorting Tier'].unique()

In [ ]:
sensor_info_df['Study'].unique()

In [ ]:
df_orig['Chemistry'].unique()

Setting

In [41]:
# training_group = ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','iBlinqFW231','iBlinqOverlay','Pre-Piv-2-training','eBlinqMayStudy','Pre-Piv-2-validation']
# training_group = ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','iBlinqFW231','iBlinqOverlay']
training_group = ['eBlinqMayStudy']
test_group = ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','iBlinqFW231','iBlinqOverlay','Pre-Piv-2-training','eBlinqMayStudy','Pre-Piv-2-validation']
# test_group = ['eBlinqMayStudy','Pre-Piv-2-validation']
all_training_channel = ['1']
training_time_window = [0.083,5]
training_chemistry = '2.6.11'
test_chemistry = '2.6.11'
training_tier = ['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide','Tier 1w-c: 0/3 CLN, 3/3 Wide', 'Tier 1w-b: 1/3 CLN, 3/3 Wide','Tier 1w-d: Missing data','Tier 1']
test_tier = ['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide','Tier 1w-c: 0/3 CLN, 3/3 Wide', 'Tier 1w-b: 1/3 CLN, 3/3 Wide','Tier 1w-d: Missing data','Tier 1']
iso_thresh = 20
params_bound = [
    [0,1e5],
    [1e-6,1],[1e-6,1],
    [0,1e3],[0,1e3],
    [1e-1,1e2],[1e-1,10],
]
col_for_optim = [
    'cur1','cur3','cur4','Chemistry','Sorting Tier',
    'time_from_start_featureGeneration','mag1_1007_featureGeneration','mag3_1007_featureGeneration','mag4_1007_featureGeneration',
    'Study','retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose'
]
ref_for_optim = 'ysi'
col_for_optim.append(ref_for_optim)

In [40]:
def glucose_model(cur,features,params):
    background_params = params['background']
    sensitivity_params = params['sensitivity']
    glucose_current,_ = background_func(cur,features,background_params)
    dermis_gluc,sensitivity_info = sensitivity_func(glucose_current,features,sensitivity_params)
    sensitivity = sensitivity_info['sensitivity']
    return dermis_gluc, sensitivity

def params_list_to_dict(params_list,channel_str):
    params = {}
    background_params = {
        'feature_schema': {'mag_1007':f'mag{channel_str}_1007'},
        'sim_slope_background': {'value': params_list[0]},
        'sim_intercept_background_start': {'value': params_list[1]},
        'sim_intercept_background_end': {'value': params_list[2]},
        'sim_intercept_background_a': {'value': 1},
        'sim_intercept_background_loc': {'value': 0},
        'sim_intercept_background_beta': {'value': 99.15500137310596},
    }
    params['background'] = background_params
    sensitivity_params = {
        'feature_schema': {'mag_1007':f'mag{channel_str}_1007'},
        'sim_intercept_sensitivity_start': {'value': params_list[3]*1e-10},
        'sim_intercept_sensitivity_end': {'value': params_list[4]*1e-10},
        'sim_intercept_sensitivity_a': {'value': 1},
        'sim_intercept_sensitivity_loc': {'value': 0},
        'sim_intercept_sensitivity_beta': {'value': 1.1632178403532043},
        'sim_slope_sensitivity_start': {'value': params_list[5]*1e-3},
        'sim_slope_sensitivity_end': {'value': params_list[6]*1e-3},
        'sim_slope_sensitivity_a': {'value': 1},
        'sim_slope_sensitivity_loc': {'value': 0},
        'sim_slope_sensitivity_beta': {'value': 82.51104301501427},
    }
    params['sensitivity'] = sensitivity_params
    return params

def mard(params_list,g,cur,features,channel_str):
    params = params_list_to_dict(params_list,channel_str)
    g_hat, _ = glucose_model(cur,features,params)    
    mard = g_hat.sub(g).abs().div(g).mean()
    return mard

def costFunc(params_list,g,cur,features,channel_str):
    params = params_list_to_dict(params_list,channel_str)
    g_hat, sensitivity = glucose_model(cur,features,params)
    error = g_hat.sub(g).div(g).mul(sensitivity).mul(1e3).abs().mean()
    return error

def callback_da(x,f,context=0):
    params_dict = params_list_to_dict(x,1)
    print(f'Cost function value is {f}, and parameters are {params_dict}')

In [42]:
df = df_orig[col_for_optim].copy()
df.rename(columns={
                'time_from_start_featureGeneration': 'time_from_start',
                'mag1_1007_featureGeneration': 'mag1_1007',
                'mag3_1007_featureGeneration': 'mag3_1007',
                'mag4_1007_featureGeneration': 'mag4_1007',
                'retro_model_output_s1_calculateGlucose': 'retro_model_output_s1',
                'retro_model_output_s3_calculateGlucose': 'retro_model_output_s3',
                'retro_model_output_s4_calculateGlucose': 'retro_model_output_s4',
                }, inplace=True)
all_results= {}
all_optim_params = {}
hypo_idx = df[ref_for_optim].le(100)
for ch_str in all_training_channel:
    print(f'Optmizing channel {ch_str}')
    this_result = []
    df[f'ad_s{ch_str}'] = df[f'retro_model_output_s{ch_str}'].sub(df[ref_for_optim]).abs()
    df[f'ard_s{ch_str}'] = df[f'ad_s{ch_str}'].div(df[ref_for_optim]).mul(100)
    df[f'iso_s{ch_str}'] = df[f'ard_s{ch_str}']
    df.loc[hypo_idx,f'iso_s{ch_str}'] = df.loc[hypo_idx,f'ad_s{ch_str}']
    col_for_optim_no_ysi_acck = list(df.columns)
    training_index = df['Study'].isin(training_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & \
                            df['Chemistry'].eq(training_chemistry) & df['Sorting Tier'].isin(training_tier)
    df_training = df.loc[training_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
    valid_training_index = df['Study'].isin(training_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & \
                            df[f'iso_s{ch_str}'].le(iso_thresh) & df['Chemistry'].eq(training_chemistry) &  df['Sorting Tier'].isin(training_tier)
    df_training_valid = df.loc[valid_training_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
    print(f'Training data has shape {df_training.shape}')
    print(f'Training data has shape {df_training_valid.shape}')
    good_data_ratio = len(df_training_valid)/len(df_training)
    print(f'Ratio of good data is {good_data_ratio}')
    valid_test_index = df['Study'].isin(test_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & \
                            df[f'iso_s{ch_str}'].le(iso_thresh) & df['Chemistry'].eq(training_chemistry)  & df['Sorting Tier'].isin(test_tier)
    df_test_valid = df.loc[valid_test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
    print(f'Test data has shape {df_test_valid.shape}')
    g_train = df_training_valid[ref_for_optim]
    features_train = df_training_valid[['time_from_start',f'mag{ch_str}_1007']]
    cur_train = df_training_valid[f'cur{ch_str}']

    g_test = df_test_valid[ref_for_optim]
    features_test = df_test_valid[['time_from_start',f'mag{ch_str}_1007']]
    cur_test = df_test_valid[f'cur{ch_str}']

    result = scipy.optimize.dual_annealing(mard,params_bound,callback=callback_da,args=(g_train,cur_train,features_train,ch_str),maxiter=int(1e3))

    this_result.append(mard(result.x,g_train,cur_train,features_train,ch_str))
    this_result.append(mard(result.x,g_test,cur_test,features_test,ch_str))

    params_dict = params_list_to_dict(result.x,ch_str)
    g_hat_train, _ = glucose_model(cur_train,features_train,params_dict) 
    g_hat_test, _ = glucose_model(cur_test,features_test,params_dict) 

    reg_train = LinearRegression().fit(g_train.values.reshape(-1,1), g_hat_train.values)
    this_result.append(reg_train.score(g_train.values.reshape(-1,1), g_hat_train.values))
    this_result.append(reg_train.coef_)
    this_result.append(reg_train.intercept_)           

    reg_test = LinearRegression().fit(g_test.values.reshape(-1,1), g_hat_test.values)
    this_result.append(reg_test.score(g_test.values.reshape(-1,1), g_hat_test.values))
    this_result.append(reg_test.coef_)
    this_result.append(reg_test.intercept_)

    all_optim_params[f'channel_{ch_str}'] = params_dict
    all_results[f'channel_{ch_str}'] = this_result
    
results_df = pd.DataFrame.from_dict(all_results)
results_df.to_csv(fr'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\optimize_time_varying_model_{ref_for_optim}_exp_indexFixed_MayTune_ch1.csv',header=True)
with open (fr'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params_{ref_for_optim}_exp_indexFixed_MayTune_ch1.p','wb') as fp:
    pickle.dump(all_optim_params,fp)    

Optmizing channel 1
Training data has shape (44042, 17)
Training data has shape (31538, 17)
Ratio of good data is 0.7160891875936606
Test data has shape (244045, 17)
Cost function value is 8.872696056408417, and parameters are {'background': {'feature_schema': {'mag_1007': 'mag1_1007'}, 'sim_slope_background': {'value': 36758.35445450246}, 'sim_intercept_background_start': {'value': 0.10598142496937366}, 'sim_intercept_background_end': {'value': 0.7239095164059982}, 'sim_intercept_background_a': {'value': 1}, 'sim_intercept_background_loc': {'value': 0}, 'sim_intercept_background_beta': {'value': 99.15500137310596}}, 'sensitivity': {'feature_schema': {'mag_1007': 'mag1_1007'}, 'sim_intercept_sensitivity_start': {'value': 7.893084292560816e-08}, 'sim_intercept_sensitivity_end': {'value': 6.405663192272187e-11}, 'sim_intercept_sensitivity_a': {'value': 1}, 'sim_intercept_sensitivity_loc': {'value': 0}, 'sim_intercept_sensitivity_beta': {'value': 1.1632178403532043}, 'sim_slope_sensitivit

In [43]:
all_optim_params

{'channel_1': {'background': {'feature_schema': {'mag_1007': 'mag1_1007'},
   'sim_slope_background': {'value': 18.757342837626382},
   'sim_intercept_background_start': {'value': 0.9768438229207428},
   'sim_intercept_background_end': {'value': 0.002064822793480901},
   'sim_intercept_background_a': {'value': 1},
   'sim_intercept_background_loc': {'value': 0},
   'sim_intercept_background_beta': {'value': 99.15500137310596}},
  'sensitivity': {'feature_schema': {'mag_1007': 'mag1_1007'},
   'sim_intercept_sensitivity_start': {'value': 9.90845395689914e-08},
   'sim_intercept_sensitivity_end': {'value': 2.792508900499859e-10},
   'sim_intercept_sensitivity_a': {'value': 1},
   'sim_intercept_sensitivity_loc': {'value': 0},
   'sim_intercept_sensitivity_beta': {'value': 1.1632178403532043},
   'sim_slope_sensitivity_start': {'value': 0.002861367305638684},
   'sim_slope_sensitivity_end': {'value': 0.0011228466835296923},
   'sim_slope_sensitivity_a': {'value': 1},
   'sim_slope_sensiti

In [ ]:
df_training_valid.to_csv(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\Final round\df_training_valid.csv',header=True)

In [ ]:
with open (r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params.p','wb') as fp:
    pickle.dump(all_optim_params,fp)

In [ ]:
with open (fr'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params_{ref_for_optim}_exp_indexFixed_All.p','rb') as fp:
    all_optim_params2 = pickle.load(fp)

In [ ]:
all_optim_params2

In [ ]:
df_training_valid['cgm_adj'].le(60).mean()

In [ ]:
df_training_valid['cgm_adj'].between(300,350,inclusive='right').mean()

In [ ]:
df_training_valid['cgm_adj'].gt(350).mean()